In [1]:
from google.colab import files

uploaded = files.upload()
# Kokeile "lakko" -datoja

Saving reddit_data_sample_test_s.parquet to reddit_data_sample_test_s.parquet
Saving reddit_data_sample_train_s.parquet to reddit_data_sample_train_s.parquet


In [ ]:
#!pip install pyarrow

In [2]:
import pandas as pd
import pyarrow

df_train = pd.read_parquet("reddit_data_sample_train_s.parquet") # s on pienempi data, 15892 riviä
df_eval = pd.read_parquet("reddit_data_sample_test_s.parquet")

#df_train = pd.read_parquet("lakko_data_sample_train_s.parquet")
#df_eval = pd.read_parquet("lakko_data_sample_test_s.parquet")

In [3]:
df_train["labels"] = df_train["labels"].astype("int")
df_eval["labels"] = df_eval["labels"].astype("int")

df_train["text"] = df_train["text"].astype("str")
df_eval["text"] = df_eval["text"].astype("str")

In [4]:
df_train.head()

,labels,text
0,1,Veeti ja Niilo siellä vaan laittaneet omat hel...
1,1,Ohan pirkkalassa valtavat ostoskeskuksetkin. J...
2,1,3A ja 3B takaisin vois kyl olla
3,1,Jalkakäytäväthän tuossa uutisen mukaan jää aur...
4,1,Mitä jos laittais lisää vaunuja niihi ratikoih...


In [5]:
df_eval.head()

,labels,text
0,1,Yksityisautoilijat jättävät sitten kiltisti au...
1,1,Voisin kuvitella että eivät vielä suunnittele ...
2,1,Juu vastasin. mielelläni autan opiskelijoita.
3,0,Miksi ihmeessä sen pitää olla kaksio? 1400e/kk...
4,1,Ainakin toinen tossa suunnilleen samoilla kohd...


In [6]:
df_train.dtypes

,0
labels,int64
text,object


In [8]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00


In [ ]:
#!pip install transformers

In [9]:
from datasets import Dataset

dataset_train = Dataset.from_pandas(df_train)
dataset_eval = Dataset.from_pandas(df_eval)

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

#import transformers
#import torch

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import AutoTokenizer, DataCollatorWithPadding

def tokenize_function(example):
    return tokenizer(example["text"], padding=True, truncation=True) # Mieti erityisesti truncationia viestidatassa

tokenized_train = dataset_train.map(tokenize_function, batched=True)
tokenized_eval = dataset_eval.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/15892 [00:00<?, ? examples/s]

Map:   0%|          | 0/3980 [00:00<?, ? examples/s]

In [12]:
tokenized_train = tokenized_train.remove_columns("text") # POISTA TÄSSÄ VAIN TEXT!
tokenized_eval = tokenized_eval.remove_columns("text")

In [13]:
tokenized_train.set_format("torch")
tokenized_eval.set_format("torch")

In [14]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_train, shuffle = True, batch_size = 8, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_eval, batch_size = 8, collate_fn=data_collator)

In [15]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 512]),
 'attention_mask': torch.Size([8, 512])}

In [16]:
import torch

torch.cuda.is_available()

True

In [17]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5) # Mitä lr tässä tarkoittaa? Learning rate?
# lr laskettu 5e-5 -> 1e-5

# Tutustu, miten määrittää oikea learning rate

In [18]:
from transformers import get_scheduler

num_epochs = 2 # Mieti tätä -> PIENENTÄMINEN VOI AUTTAA SAMOJEN AROVJEN ENNUSTAMISEEN, LASKE ESIM. KAHTEEN
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [19]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=

In [20]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

# Aika-arvio n. 5 h
model.train() # Opettele lisäämään checkpointien tallennus tähän.
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

# Aikaa kului 52:16

  0%|          | 0/3974 [00:00<?, ?it/s]

In [ ]:
#import os
#os.makedirs("/content/drive/My Drive/Colab Notebooks/models") # content tästä polusta pois??
model.save_pretrained("/content/drive/My Drive/Colab Notebooks/models") # LATAA HETI!!!
# Tämä näyttäisi toimivan
# MUTTA MISTÄ MALLI LÖYTYY DRIVESTA TÄMÄN JÄLKEEN???
# Models-kansio näkyy Colabissa muttei Drivessa
# Käytä Downloadia file explorerissa???

# Laita evaluaten tiedot ylös
# ja tarkista, tuottaako evaluate samat tulokset mallin uudelleenlatauksen jälkeen

In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
import evaluate

metric = evaluate.load("accuracy") # kts. evaluate.list_evaluation_modules() ja https://huggingface.co/evaluate-metric
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()
# accuracy 0.6844

{'accuracy': 0.6844221105527638}

In [ ]:
# Siirrä tästä eteenpäin uuteen notebookiin

# Uudessa notebookissa:
# Tarkista, että accuracy alkuperäistä vastaava
# Tutki, miten saat siirrettyä ennustearvot test-dataframeen
# Jos onnistuu: miten erotella hoidolle yhteinen ja erillinen varianssi?


from transformers import AutoModelForSequenceClassification


# Lataaminen pitäisi onnistua tällä
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/My Drive/Colab Notebooks/models")

# Tarkista lataamisen jälkeen, että accuracy vastaa trainingin jälkeistä
# Pitäisi olla .6844

In [ ]:
#!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00


In [ ]:
import evaluate

# Huomaa, että tässäkin saattaa kestää CPU:lla kauan
metric = evaluate.load("accuracy") # kts. evaluate.list_evaluation_modules() ja https://huggingface.co/evaluate-metric
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

KeyboardInterrupt: 